In [13]:
import csv
import numpy as np
import re
import os
with open('/Users/yao/Desktop/band offset/band_offset_collect.csv', 'r') as f:
    reader = csv.reader(f)
    csvlist = list(reader)
    offset_dict = get_all_offset_pairs(csvlist)
    gap_dict = get_all_compounds_gap(csvlist)
    comp_set = get_all_compounds(csvlist)
    del_set = set(['CH3NH3PbBr3', 'CH3NH3PbCl3', 'CH3NH3PbI3', 'CH3NH3PbIBr2', 'HC(NH2)2PbI3',
               'Er2O3', 'Fe2O3', 'Gd2O3', 'HfO2', 'InGaZnO4', 'CuInGaSe2', 'InSnO', 'Lu2O3',
               'NiO', 'NiOx', 'Pr2O3', 'Tm2O3'])
    desired_set = comp_set-del_set
    desired_list = [i for i in desired_set]
#os.remove('/Users/yao/Desktop/band offset/band_offset_collect.csv')
with open('/Users/yao/Desktop/band offset/band_offset_matrix.csv', 'w') as f:
    counter = generate_offset_matrix(desired_list, offset_dict, f)
counter

170.0

In [14]:
with open('/Users/yao/Desktop/band offset/gap.csv', 'w') as f:
    for line in gap_dict:
        f.write(line)
        f.write(',')
        f.write(str(gap_dict[line]))
        f.write('\n')
        
with open('/Users/yao/Desktop/band offset/offset_list.csv', 'w') as f:
    for item in offset_dict:
        f.write(item.split('/')[0])
        f.write(',')
        f.write(item.split('/')[1])
        f.write(',')
        f.write(str(offset_dict[item]))
        f.write('\n')

In [24]:
calculated = set(['C', 'Si', 'Ge', 'SiC', 
             'Si3N4', 'AlN', 'GaN', 'InN', 
             'AlP', 'GaP', 'InP',
             'AlAs', 'GaAs', 'InAs', 
             'AlSb', 'GaSb', 'InSb', 
             'MgO', 'ZnO', 
             'ZnS', 'CdS', 'SnS', 'CuInS2', 'Cu2ZnSnS4', 
             'ZnSe', 'CdSe', 'CuGaSe2', 'ZnTe', 'CdTe', 'HgTe', 
             'SiO2','GeO2','SnO2', 
             'Ga2O3', 'In2O3', 'In2S3', 
             'Cu2O', 'TiO2', 
             'SrTiO3', 'BaTiO3',
             'CaF2', 'CdF2', 'Te'])
compounds = [i for i in (desired_set-calculated)]

with open('/Users/yao/Desktop/band offset/gap3_tmp.csv', 'w') as f:
    for comp in compounds:
        f.write(comp)
        f.write(',')
        if comp in gap_dict:
            f.write(str(gap_dict[comp]))
        else:
            f.write('NA')
        f.write('\n')

In [38]:
completed_set = set(['AlN', 'AlP', 'AlAs', 'AlSb',
                 'GaN', 'GaP', 'GaAs', 'GaSb',
                 'InN', 'InP', 'InAs', 'InSb', 
                 'ZnO', 'ZnS', 'ZnSe', 'ZnTe', 
                 'CdS', 'CdSe', 'CdTe'])
remain_set = desired_set - completed_set
remain_list = [i for i in remain_set]
#print(remain_list)

compA = 'CdTe'
for compB in desired_list:
    if compA+'/'+compB in offset_dict:
        print(compB)

Te
HgTe
ZnTe
ZnS
InSb
GaAs
TiO2
PbTe
In2O3
CdS
ZnO


In [7]:
def generate_offset_matrix(desired_list, offset_dict, filename):
    """generate a csv file that has a matrix for offset pairs 
    with desired compounds of x/y axis
    return the number of pairs"""
    counter = 0
    writer = csv.writer(filename)
    writer.writerow(['']+desired_list)
    for compoundA in desired_list:
        row_list = [compoundA]
        for compoundB in desired_list:
            if compoundA+'/'+compoundB in offset_dict:
                row_list.append(str(offset_dict[compoundA+'/'+compoundB]))
                counter = counter+1
            else:
                row_list.append('')
        writer.writerow(row_list)
    return counter/2

In [8]:
def get_all_compounds(csvlist):
    """return a set of all compounds"""
    comp_list = []
    for i in range(1, len(csvlist)):
        csvlist[i][0] = csvlist[i][0].replace(" ", "")
        compoundA = csvlist[i][0].split('/')[0]
        compoundB = csvlist[i][0].split('/')[1]
        comp_list.append(compoundA)
        comp_list.append(compoundB)
    return set(comp_list)

In [9]:
def get_all_compounds_gap(csvlist):
    """return a dict of all compounds with gap"""
    gap_dict = {}
    gap_dict_tmp = {}
    for i in range(1, len(csvlist)):
        csvlist[i][0] = csvlist[i][0].replace(" ", "")
        compoundA = csvlist[i][0].split('/')[0]
        compoundB = csvlist[i][0].split('/')[1]
    
        if csvlist[i][3] != '':
            gapA = [float(j) for j in csvlist[i][3].split(',')]
            if compoundA not in gap_dict_tmp:
                gap_dict_tmp[compoundA] = gapA
            else:
                gap_dict_tmp[compoundA] = gap_dict_tmp[compoundA] + gapA
        
        if csvlist[i][4] != '':
            gapB = [float(j) for j in csvlist[i][4].split(',')]
            if compoundB not in gap_dict_tmp:
                gap_dict_tmp[compoundB] = gapB
            else:
                gap_dict_tmp[compoundB] = gap_dict_tmp[compoundB] + gapB
                
    for compound in gap_dict_tmp:
        gap_dict[compound] = np.mean(gap_dict_tmp[compound])
        
    return gap_dict
        
    

In [10]:
def get_all_offset_pairs(csvlist):
    """return a dict of all valence band offset pairs, twice.
    {compoundA/compoundB: valence band offset}
    Positive values mean that the valence band maximum of 
    semiconductor A is higher than that of semiconductor B
    """
    offset_dict = {}
    for i in range(1, len(csvlist)):
        csvlist[i][0] = csvlist[i][0].replace(" ", "")
        compoundA = csvlist[i][0].split('/')[0]
        compoundB = csvlist[i][0].split('/')[1]
        offset = np.mean([float(j) for j in csvlist[i][1].split(',')])
        
        if compoundA+'/'+compoundB in offset_dict:
            offset_dict[compoundA+'/'+compoundB] = \
            (offset_dict[compoundA+'/'+compoundB] + offset)/2
            print(compoundA, compoundB)
        else:
            offset_dict[compoundA+'/'+compoundB] = offset
            
        offset_dict[compoundB+'/'+compoundA] = 0-offset_dict[compoundA+'/'+compoundB]
        
    return offset_dict
        